In [1]:
import json
from tqdm import tqdm
from glob import glob

files = glob('texts-theblackcat102.jsonl*.splitted.requested')
files

['texts-theblackcat102.jsonl07.splitted.requested',
 'texts-theblackcat102.jsonl06.splitted.requested',
 'texts-theblackcat102.jsonl04.splitted.requested',
 'texts-theblackcat102.jsonl05.splitted.requested',
 'texts-theblackcat102.jsonl09.splitted.requested',
 'texts-theblackcat102.jsonl08.splitted.requested',
 'texts-theblackcat102.jsonl03.splitted.requested',
 'texts-theblackcat102.jsonl00.splitted.requested',
 'texts-theblackcat102.jsonl01.splitted.requested',
 'texts-theblackcat102.jsonl02.splitted.requested']

In [3]:
mapping = {}
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            mapping[data['src']] = data['r']['result']

43689it [00:00, 129273.33it/s]
42997it [00:00, 175983.72it/s]
43511it [00:00, 181184.95it/s]
43855it [00:00, 182062.68it/s]
43692it [00:00, 146429.25it/s]
44009it [00:00, 174850.41it/s]
43597it [00:00, 174077.72it/s]
43263it [00:00, 177637.37it/s]
42995it [00:00, 124379.58it/s]
43013it [00:00, 178624.43it/s]


In [4]:
len(mapping)

342471

In [5]:
data = []
with open('train.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))

In [6]:
len(data)

50496

In [8]:
for i in range(len(data)):
    for k in range(len(data[i]['conversations'])):
        ms = mapping.get(data[i]['conversations'][k]['text'])
        data[i]['conversations'][k]['text_ms'] = ms

In [9]:
data[1]

{'conversations': [{'user': 'human',
   'text': 'Web search results:\n\n[1] "Whatever the reason, starting a membership organization can brighten, serve, or elevate your community. In this guide, we will share six simple steps to get your idea off the ground. Step 1: Define Your Club. Step 2: Create Your Club Structure. Step 3: Organize Your Leadership. Step 4: Figure Out Your Finances."\nSource: https://www.springly.org/en-us/blog/how-to-start-a-members-only-club/\n\n[2] "A successful membership club is one that can offer its members experiences, content, and/or rewards they wont find elsewhere. But getting to this point requires a few key steps—so start taking notes. 1. Decide what kind of club you want to create. The first step in starting a membership club is identifying its purpose and mission."\nSource: https://membershipworks.com/how-to-start-a-membership-club/\n\n[3] "1. Air Amsterdam. AIR opened its doors in Amsterdam in 2010, which is famous for having a vibrant nightlife in 

In [10]:
with open('gather-theblackcat102.jsonl', 'w') as fopen:
    for d in data:
        fopen.write(f'{json.dumps(d)}\n')

In [10]:
import random

In [11]:
all_texts = []
for i in tqdm(range(len(data))):
    texts = []
    for c in data[i]['conversations']:
        if c['from'] == 'human':
            t = '<manusia>: '
            t += c['value_ms'] if random.random() < 0.6 and c['value_ms'] else c['value']
        else:
            if c['value_ms'] is None:
                break
            t = '<bot>: ' + c['value_ms']
        texts.append(t)
    if len(texts) % 2 != 0:
        texts = texts[:-1]
    if not len(texts):
        continue
    if not texts[0].startswith('<manusia>'):
        continue
    all_texts.append('\n'.join(texts))

100%|█████████████████████████████████| 94145/94145 [00:00<00:00, 276610.25it/s]


In [12]:
len(all_texts)

46657

In [13]:
with open('vicuna-all_texts.jsonl', 'w') as fopen:
    for t in all_texts:
        fopen.write(f'{json.dumps(t)}\n')